In [1]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import CplexOptimizer

mod = QuadraticProgram("test")
mod.continuous_var(name="x1", lowerbound=0)
mod.continuous_var(name="x2", lowerbound=0)
mod.continuous_var(name="x3", lowerbound=0)

mod.minimize(
    quadratic={
        ("x1", "x1"): 3,
        ("x2", "x2"): 2,
        ("x3", "x3"): 5,
        ("x1", "x2"): -4,
        ("x1", "x3"): -2,
        ("x2", "x3"): -2,
    }
)

mod.linear_constraint(linear={"x1": 3, "x2": 5, "x3": 2}, sense=">=", rhs=10)
mod.linear_constraint(linear={"x1": 3, "x3": 5}, sense="<=", rhs=15)

print(mod.prettyprint())
# solve the problem

optimizer = CplexOptimizer()
result = optimizer.solve(mod)
print(result)

Problem name: test

Minimize
  3*x1^2 - 4*x1*x2 - 2*x1*x3 + 2*x2^2 - 2*x2*x3 + 5*x3^2

Subject to
  Linear constraints (2)
    3*x1 + 5*x2 + 2*x3 >= 10  'c0'
    3*x1 + 5*x3 <= 15  'c1'

  Continuous variables (3)
    0 <= x1
    0 <= x2
    0 <= x3

fval=0.10905125502080393, x1=0.9814612909979425, x2=1.2322791764290346, x3=0.4471101437824319, status=SUCCESS


In [11]:
from qiskit_optimization.applications import Maxcut
import numpy as np
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms.optimizers import SPSA, COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms import SamplingVQE, QAOA
from qiskit.primitives import Sampler

algorithm_globals.random_seed = 123
seed = 10598

w = np.array(
    [
        [0.0, 1.0, 1.0, 1.0, 0.0],
        [1.0, 0.0, 1.0, 0.0, 0.0],
        [1.0, 1.0, 0.0, 1.0, 1.0],
        [1.0, 0.0, 1.0, 0.0, 1.0],
        [0.0, 0.0, 1.0, 1.0, 0.0],
    ]
)

max_cut = Maxcut(w)
qp = max_cut.to_quadratic_program()
qubitOp, offset = qp.to_ising()
print(qp.prettyprint())

# using SamplingVQE
# optimizer = SPSA(maxiter=300)
# ry = TwoLocal(qubitOp.num_qubits, "ry", "cz", reps=5, entanglement="linear")
# vqe = SamplingVQE(sampler=Sampler(), ansatz=ry, optimizer=optimizer)

# run SamplingVQE
# result = vqe.compute_minimum_eigenvalue(qubitOp)

# using QAOA
optimizer = COBYLA()
qaoa = QAOA(sampler=Sampler(), optimizer=optimizer, reps=2)

# run QAOA
result = qaoa.compute_minimum_eigenvalue(qubitOp)

# print results
x = max_cut.sample_most_likely(result.eigenstate)

print("energy:", result.eigenvalue.real)
print("time:", result.optimizer_time)
print("max-cut objective:", result.eigenvalue.real + offset)
print("solution:", x)
print("solution objective:", qp.objective.evaluate(x))

Problem name: Max-cut

Maximize
  -2*x_0*x_1 - 2*x_0*x_2 - 2*x_0*x_3 - 2*x_1*x_2 - 2*x_2*x_3 - 2*x_2*x_4
  - 2*x_3*x_4 + 3*x_0 + 2*x_1 + 4*x_2 + 3*x_3 + 2*x_4

Subject to
  No constraints

  Binary variables (5)
    x_0 x_1 x_2 x_3 x_4

energy: -0.9744508289537032
time: 21.40344214439392
max-cut objective: -4.474450828953703
solution: [1 0 1 0 0]
solution objective: 5.0
